* Resnet
    * Jeremy added an 'include_top' parameter which can be set to False if you don't need the imagenet classification layers so you don't have to do model.pop()
    * Everything in this tutorial will not train the conv layers so all the examples will train quick. Because: J experience is that with photos it's almost never usefull to retrain the convolutional layers
    * With Batch, flat, drop, dense, batch, drop, dense, batch, drop, dense and 50s training, you get a 2nd place in catsanddogs
    * With globalavgpool, drop, dense you get first in 3s training 98.7
    * With bigger pictures (224->400) you go off the charts, 99.3 in 350s
    * Resnet is incredibly accurate for catsdogs, probably because it has knowledge about cnd's due to imagenet training.
    * Resnet architecture
        * Resnet is similar to vgg but als does layer skipping
        * Where vgg is a list of conv, batchnorm and activation, resnet does the same but then every three layers, the input of the first layer is added (merge x, input like identity) in parallel after the third layer. This happens multiple times, multiple identity blocks.
        * hidden_state(t+1) = conv(conv(conv(hidden_state(t)))) + hidden_state(t)
        * Reasons why it works:
            * 1: You can make much deeper networks because backprop through identity is easy.
            * BUT, turned out to also be able to make much wider networks and have even better results
            * 2: Essentially applying 'boosting'. You could rewrite the previous formula as h(t+1)-h(t) = c(c(c(h(t)))) = R(t). So h(t+1)-h(t) = R(h(t)). R is the improvement of the hidden layer at a previous time step. 
        * Per identity block the dim's stay the same. Outside of the block you do dim reduction ie maxpooling
        * Resnet uses no dropout, you could add it if you have a small amount of data.
        * GlobalAveragePooling2D. Pooling is taking the max/avg of a bunch of neurons and then passing that through. Global means that the stride is 0 because the pooling is done over the entire previous layer/filter. If you have 10 filters with size 3x3, after global avg pooling you have 10 avg's.
            * Difference max and avg maxpooling: max takes max of subblock, avg takes avg. That's the output for the next layer.
            * Difference global and local pooling: example: previous layer is identity block with filters. global; do avg/max for the entire image, lose all location data but use previous filters as charachteristics, you have less parameters. local; do avg/max with strides, you get localization data but with more parameters.
        * Resnet is trained with a avgpooling layer at the end so the last idetity block output features that are good to retrain with some avgpooling.
        * Resnet doesnt need dropout. When using globalavgpooling you have less parameters so you need less dropout. Normally and in vgg, there are dense layers and dense layers have many parameters. In Resnet you have globalavgpooling which is kind of selecting dogness.
* Transfer learning with vgg
    * Transfer learning is copying a pretrained network and apply it to your own dataset
    * mostly done with vgg because the idea is to have increasing difficulty of features
    * Matt zeiler, Jason yusinski
* Fisheries competition
    * Data Leakage: the model uses wrong data for classifying
        * When a model is trained on some dataset, it is possible that it trains on data you didn't intend it to train on. In this case, you train on pictures from non changing location on the boat so the net can see which boat the data is from. Some boats fish in different areas with different fish. The model learns this too although you might not want it to.
        * This is often already encoded in the original data. ie fish competition, recognize the boat from the size of the picture because boats have different sizes. But the model already sees this from the pictures.
        * Often looking for metadata is a WASTE OF TIME. The original data usually already contains all info.
        * Adding use of metadata, think about how this generalizes towards the future use of you model; new pictures, you don't have the class
    * Multi output network
        * For the fisheries competition, someone added boxes around the fish for each picture and made that information available
        * We can now add an output in the functional api and train that output for those boxes. Due to this, the model is trained, not only to detect fish kinds but also where these fish are in the picture. This gives better results.
        * This is fascinating, just train to find the fish, additionally to classifiying improves and stabalizes the results of the bare fish prediction part of the model
    * GENERAL: learn from doing Kaggle competitions. If you're first and then not anymore, you know the others found something that you didn't think of. Stop in time with fucking around with hyper parameters and minor architectual details because it only helps so much. Stop and think of something new.
        * Learn most by playing around with competitions
* Part 2
    * input sizes
        * Only layer that is dependent of the input size of the image/data is dense layers.
        * Conv layer just moves the filter over the input. If the image is bigger, the output will be bigger but no problem.
        * MaxPooling doesn't have weights.
        * Batchnorm, cares about the weights of the previous layer
        * If you strip vgg of dense layers, you can use any size of input. You could add some other conv layers this is a Fully Convolutional Net (FCN).
    * FCN: For instance; take vgg and split of dense part. Than add some conv, batchnorm, maxpool etc and end with amount of filters that are categories so that you train for particular categories.
        * If you do this, you can look at the output of the last conv layer which shows areas of particular fishness with a heatmap. If you remove pooling, you get a higher resolution heat map.
        * You can learn something from it; if the heatmap points at seemingly non relevant areas of the picture, probably you have some form of data leakage.
    * Find bounding boxes
        * You could take the biggest area on the heatmap and draw a box around it.
* Inception net
    * For each inception block: use different sizes of conv filters and and avgpooling layer and concatinate them before the end of the block.
    * Like Resnet, it hasn't been studied in terms of it's transfer learning capabilities.
    * Last year's imagenet winner was Inception plus Resnet
* J: vast majority of solutions was CNN's. Most challenging problems such as speech rec and language translations are RNN's. Timeseries are also RNN's. 
1:30 - 1:44 niet gekeken, python implementation van Theano implementatie van RNN. Omdat RNN's het moeijlijkst zijn is dit een goeie oefening.
1:44 LSTM en GRU. Only GRU because it's simpler and better.
1:54 next level intuition:
* Understand why stuff works; why does Resnet work, why does FCN work, etc. Try, look at intermediate results, understand what is going on.
